### one-hot 编码

one-hot 编码用来把一个类别标签转化为概率分布

如果是共有 5 个类别的话

例如标签 标签 tensor([1])

```python
转化为 tensor([
    [0, 1, 0, 0, 0]
])  # 第 1 号类别的概率为 1，其余为 0
```

In [127]:
import torch
import torch.nn.functional as F

vec = torch.arange(3)
F.one_hot(vec, num_classes=5)  # 类别的个数为 5

tensor([[1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0]])

In [128]:
mat = torch.tensor([
    [1, 2, 3],
    [4, 5, 6]
])

# batch_size, num_steps,  num_classes
print(F.one_hot(mat, 7).shape)

# num_steps, batch_size, num_classes
# 每次拿到所有样本的同一个时间步上的数据集
print(F.one_hot(mat.T, 7).shape)

torch.Size([2, 3, 7])
torch.Size([3, 2, 7])


In [129]:
# one-hot 编码逆向转换

torch.argmax(torch.tensor([
    [0, 0, 1, 0],
    [0, 1, 0, 0],
    [0, 0, 0, 1],
]), dim=1)

tensor([2, 1, 3])

#### 创建虚拟数据序列

虚拟的数据集的词元是数值 0-19
长度为 1000

In [130]:
from random import randint

seq = []

for i in range(1000 // 10):
    # 每次生成长度为 10 的连续序列
    start = randint(0, 10)  # [0, 10] 闭区间随机一个数值
    for j in range(start, start+10):
        seq.append(j)

In [131]:
len(seq), min(seq), max(seq)

(1000, 0, 19)

#### 构建数据集

In [132]:
T = len(seq)  # 序列数据 seq 的长度
num_steps = 20  # 时间步长
num_classes = 20  # 有多少个种类的词元;用于生成 ont-hot 编码

In [133]:
dtype = torch.int64

x = torch.tensor(seq, dtype=dtype)
X = torch.zeros((T-num_steps, num_steps), dtype=dtype)
Y = torch.zeros((T-num_steps, num_steps), dtype=dtype)

In [134]:
for i in range(num_steps):
    X[:, i] = x[i:T-num_steps+i]
    Y[:, i] = x[i+1:T-num_steps+i+1]

In [135]:
X[0, :11]

tensor([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14,  0])

In [136]:
Y[0, :11]

tensor([ 6,  7,  8,  9, 10, 11, 12, 13, 14,  0,  1])

#### 创建数据迭代器

In [137]:
batch_size = 50  # 批量大小

sample_epochs = X.shape[0] // batch_size
num_samples = sample_epochs * batch_size

from torch.utils import data
dataset = data.TensorDataset(X[:num_samples, :], Y[:num_samples, :])
diter =data.DataLoader(dataset=dataset, batch_size=batch_size)

In [138]:
for _X, _Y in diter:
    # one_hot_X = F.one_hot(_X, num_classes)
    # print(one_hot_X.shape)
    print(_X.shape, ' and ', _Y.shape)

torch.Size([50, 20])  and  torch.Size([50, 20])
torch.Size([50, 20])  and  torch.Size([50, 20])
torch.Size([50, 20])  and  torch.Size([50, 20])
torch.Size([50, 20])  and  torch.Size([50, 20])
torch.Size([50, 20])  and  torch.Size([50, 20])
torch.Size([50, 20])  and  torch.Size([50, 20])
torch.Size([50, 20])  and  torch.Size([50, 20])
torch.Size([50, 20])  and  torch.Size([50, 20])
torch.Size([50, 20])  and  torch.Size([50, 20])
torch.Size([50, 20])  and  torch.Size([50, 20])
torch.Size([50, 20])  and  torch.Size([50, 20])
torch.Size([50, 20])  and  torch.Size([50, 20])
torch.Size([50, 20])  and  torch.Size([50, 20])
torch.Size([50, 20])  and  torch.Size([50, 20])
torch.Size([50, 20])  and  torch.Size([50, 20])
torch.Size([50, 20])  and  torch.Size([50, 20])
torch.Size([50, 20])  and  torch.Size([50, 20])
torch.Size([50, 20])  and  torch.Size([50, 20])
torch.Size([50, 20])  and  torch.Size([50, 20])


#### 浅尝 RNN 层的前向传播

In [139]:
import torch
from torch import nn
import torch.nn.functional as F

In [140]:
num_hiddens = 256  # rnn 隐藏层的输出个数 - 也是隐藏单元个数
num_classes = 20  # rnn 隐藏层的输入个数 - 也是 ont-hot 编码的长度 - 代表词元种类个数

In [141]:
# 输入为 num_classes
# 输出为 num_hiddens
# 1 层 rnn 堆叠
rnn_layer = nn.RNN(num_classes, num_hiddens, 1)

#### 创建隐状态

每一个样本的前向传播过程都会产生一个隐状态，因此 $H$ 要有 batch_size 行，每一行都是前向传播过程中产生的一个隐状态, 而每个隐状态其实只是中间层的输出，所以有 num_hiddens 列

通过 rnn 层输出的是 new_H 也就是 new_state

In [142]:
# size = (隐藏层个数，批量大小，神经元数量)
# 在下方的测试中仅含有 1 个数据样本
state = torch.zeros(size=(1, 1, num_hiddens))
state.shape

torch.Size([1, 1, 256])

In [143]:
# (num_steps, batch_size, num_classes)

# 创建一个 x_t 样本用于测试 rnn_layer 层的前向传播
# 样本数量为 1 序列长度为 3
x_t = F.one_hot(torch.tensor([
    [1],
    [3],
    [4]
]), 20).float()
x_t.shape

torch.Size([3, 1, 20])

In [144]:
# pred 是每一个输入的 词元 对应的输出值 形状与 x_t 相同
# new_state 是每句话中最后一个词元在每层 rnn 中对应的输出值
pred, new_state = rnn_layer(x_t, state)
pred.shape, new_state.shape

(torch.Size([3, 1, 256]), torch.Size([1, 1, 256]))

In [145]:
# 第 0 号样本在第 3 号个词元对应的输出值的前 7 个值
# pred 是每个词元扔进去后在 rnn 层产生的输出，这样就不需要手动来把每个词元丢进去了
pred[2][0][:7]

tensor([ 0.0989,  0.1009, -0.0072, -0.1038,  0.0413,  0.1090,  0.0167],
       grad_fn=<SliceBackward0>)

In [146]:
# 顺序取索引
# 第 1 号层的第 0 号样本的最后一个词元的输出值的前 7 个值
# new_state[1][0][:3]
new_state[0][0][:3]

tensor([ 0.0989,  0.1009, -0.0072], grad_fn=<SliceBackward0>)

#### 矩阵的展开逻辑

从内层的维度向外层逐层展开

时间步数，批量大小，one-hot长度

第 0 步中的 第 0 号样本 的 one-hot 编码

第 0 步中的 第 1 号样本 的 one-hot 编码

...

以时间步作为分组排列在二维矩阵上

In [147]:
# 时间步数，批量大小
data = torch.tensor([
    [1, 4],
    [2, 5],
    [3, 6]
])

data

tensor([[1, 4],
        [2, 5],
        [3, 6]])

In [148]:
data = F.one_hot(data, 7)
data

tensor([[[0, 1, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 1, 0, 0]],

        [[0, 0, 1, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 1, 0]],

        [[0, 0, 0, 1, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 1]]])

In [149]:
data.reshape(-1, data.shape[-1])

tensor([[0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1]])

#### 构建 RNN 网络

In [150]:
class RNNModel(nn.Module):
    def __init__(self):
        super().__init__()
        
        # one-hot 编码长度 就是 20
        # 隐藏单元个数
        # 几层 rnn 堆叠
        self.rnn = nn.RNN(20, 256, 1)

        # 使用线性层将其转化为概率分布输出
        self.linear = nn.Linear(256, 20)

    def forward(self, inputs, state):
        X = F.one_hot(inputs.T.long(), 20)
        X = X.to(torch.float32)
        Y, state = self.rnn(X, state)
        output = self.linear(Y.reshape(-1, Y.shape[-1]))
        return output, state

    def begin_state(self, batch_size):
        # rnn 网络堆叠层数，批量大小，隐藏单元个数
        return torch.zeros(size=(1, batch_size, 256))

In [151]:
net = RNNModel()
state = net.begin_state(50)

loss = nn.CrossEntropyLoss()

updater = torch.optim.SGD(net.parameters(), lr=0.01)

for i in range(50):
    for X, Y in diter:
        y = Y.T.reshape(-1)
        # 因为 state 连接了之前的小批量数据 X 和 Y 前向运算的计算图
        # 所以在进行新的一轮前向传播计算损失 然后 计算梯度的时候
        # 要丢弃之前产生的 计算图，因此要使用 state.detach_() 来实现
        # detach_ 是一个 inplace 方法
        state = net.begin_state(batch_size=batch_size)

        y_hat, state = net(X, state)
        l = loss(y_hat, y.long()).mean()
        updater.zero_grad()
        l.backward()
        updater.step()

In [152]:
def predict(prefix, num_preds, net, device):  #@save
    """
    在prefix后面生成新字符
    num_preds: 后续要生成多少个 词（本节的次元是 字符）
    """
    # batch_size = 1 ==> 对一个字符串做预测
    state = net.begin_state(batch_size=1)
    outputs = [int(prefix[0])]  # 把 prefix[0] 对应的 数值映射 放进 outputs 中
    get_input = lambda: torch.tensor([outputs[-1]], device=device).reshape((1, 1))
    
    for y in prefix[1:]:  # 预热期 ==> 把 prefix 中的信息初始化到 state 中
        # y 是从 prefix 的 第1号 元素开始取值的
        # 当 y 取 prefix[1] 的时候，此时 get_input() 得到的输入是 prefix[0] 
        # 刚好是错开的，是没有问题的
        # 之所以 state 也是网络的输入参数
        # 就是使用 state 保存了 prefix 的信息，而 W_hh 是帮助生成 state 用的
        inputs = get_input()
        _, state = net(inputs, state)
        outputs.append(int(y))

    # 从这儿开始是预测 prefix 后边的字符
    # 每次都用上一次输出的值作为下一次输入的值
    # 上一次的输出值是一个时间步上的值，而不是一个序列
    # 这刚好和在训练模型时的思路是一致的，使用了时间序列中的
    # 某一个时间步上的值的 one-hot 编码作为输入
    # 因此这里的 RNN 在真正预测的核心部分，它的输入依旧是时间序列中的
    # 某个时间步的值，但用 ont-hot 值来表达
    for _ in range(num_preds):  # 预测num_preds步
        y, state = net(get_input(), state)
        outputs.append(int(y.argmax(dim=1).reshape(1)))

    # 把 index 的值 转化成其 对应的 词（这里是字符）
    return outputs

In [153]:
predict('234', 5, net, torch.device('cpu'))

[2, 3, 4, 5, 6, 7, 8, 9]